In [1]:
from sentiment import *

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/vedantdasswain/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vedantdasswain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Users/vedantdasswain/Documents/PostDoc/Projects/empathetic_llm/propilot.nosync/venv311/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
test_queries = [
    "Sorry? That's all you've got? A simple \"sorry\" won't fix the mess of a stay I had. What are you going to do about it?",
    "Oh, you \"hear\" me? That's just great. Listening is one thing, but I want action, not just words! What are you going to do about it?.",
    "Well, you better make it quick! And it better be the best room you have, or there will be even more complaints coming your way.",
    "The room that I booked at your hotel was not what was advertised. It was dirty and had a musty smell. I am very disappointed and will not be staying here again.",
    "I appreciate your apology. Could you please let me know what steps can be taken to address this issue?",
    "Thank you for offering to switch me to a clean room. Can you ensure that the new room will be in a better condition than the first one?",
    "That sounds great, thank you. Could you also let me know how long it will take to prepare the new room?",
    "I understand it may take some time, but spending the entire day waiting for a new room is quite inconvenient. Is there any way to expedite this process, or perhaps offer some form of compensation for the inconvenience caused?",
    "That would be appreciated. What form of compensation are you considering?"
]

In [3]:
for query in test_queries:
    print(query)
    print("NLTK:", analyze_sentiment_nltk(query))
    print("TextBlob:", analyze_sentiment_textblob(query))
    print("Transformers:", analyze_sentiment_transformer(test_queries[0]))
    print()

Sorry? That's all you've got? A simple "sorry" won't fix the mess of a stay I had. What are you going to do about it?
NLTK: Slightly Positive
TextBlob: Slightly Negative
Transformers: Very Negative

Oh, you "hear" me? That's just great. Listening is one thing, but I want action, not just words! What are you going to do about it?.
NLTK: Positive
TextBlob: Positive
Transformers: Very Negative

Well, you better make it quick! And it better be the best room you have, or there will be even more complaints coming your way.
NLTK: Very Positive
TextBlob: Positive
Transformers: Very Negative

The room that I booked at your hotel was not what was advertised. It was dirty and had a musty smell. I am very disappointed and will not be staying here again.
NLTK: Very Negative
TextBlob: Very Negative
Transformers: Very Negative

I appreciate your apology. Could you please let me know what steps can be taken to address this issue?
NLTK: Positive
TextBlob: Neutral
Transformers: Very Negative

Thank you 

# Agent Testing

## Updates:
- Customer agent now uses GPT-4o to generate responses
- Historical context for support agents now summarizes the chat history
- Updated prompts for info cue agent to avoid full responses and leverage summary history
- Updated the trouble agent to provide actionable items, avoid general suggestions, leveraging summary history, and send cues as a list
- Updated reframing agent:
    - Situation has been completely modified. Originally, it was a summary of the complaint. Now, it includes the customer's behavior towards the representative.
    - Thought has been updated to capture the ego-threat or emotional state of the customer and includes better examples.  These thoughts are then sent for rephrasing.
    - Reframe has also been updated to capture empathetic, acitonable, and specific reframes. These thoughts are then sent for rephrasing.

In [2]:
from agents import *

sender_agent = mAgentCustomer()
info_agent = mAgentInfo()
trouble_agent = mAgentTrouble()

In [3]:
def get_history_first():
    chat_history = [AIMessage(content="                AmericanAir I have been trying to get a refund for my canceled flight for over a month now. Every time I call, I am told that it will be processed soon but it has been weeks and I still haven't received anything. This is unacceptable and frustrating. ")]
    return chat_history

def get_history_second():
    chat_history = [AIMessage(content="                AmericanAir I have been trying to get a refund for my canceled flight for over a month now. Every time I call, I am told that it will be processed soon but it has been weeks and I still haven't received anything. This is unacceptable and frustrating. "),
                    HumanMessage(content="sorry for your bad experience, could you please provide your order ID?")]
    return chat_history

prompt = "sorry for your bad experience, could you please provide your order ID?"

In [4]:
result = sender_agent.invoke({"input": prompt, "chat_history": get_history_first(), "civil": 0})
response = result
print(response)

                Why do you need my order ID again? Can't you just look it up with my name or something? Fine, it's 12345. Get it done already!


**Sample Response (POST-UPDATE)** : "Are you serious? You should have all my information lready. It's 12345678. Do your job and fix this mess."

In [33]:
historic_context = sender_agent.get_historical_context_chain().invoke({'question':prompt, "chat_history": get_history_first()})

**Sample Historic Context**: 'Could you please provide your order ID for the refund request regarding your canceled flight?'

### Historic Context

In [34]:
historic_context = info_agent.get_historical_info_context_chain().invoke({'domain': "airlines",'complaint':response, "chat_history": get_history_second()})

In [35]:
historic_context

'My order ID is 123456789. Please process my refund immediately.'

**Sample Historic Context:**
'My order ID is 12345678. Please resolve this issue promptly.'

Output does not always reflect historic context, but rather only the most recent information provided by the customer. The invocation used here already leverages GPT-4o. Therefore, we need to adjust the prompts to reflect the actual context.
Another problem is that the support panes need historical summaries, not just rephrases. 

In [5]:
client_completion = lcai.AzureOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    openai_api_version="2024-05-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name="PROPILOT",
    model_name="gpt-4o",
)

def get_historical_context_chain():
    contextualize_q_system_prompt = """
    Your role is to ensure that {message} can be understood without the chat history.\
    
    The chat history contains an online conversation between a customer and a support representative.\
    The {message} is {sender}'s latest response in the chat.\
    
    Summarize the chat history in a way that provides context for the {message}.\
    """
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{sender}:{message}"),
        ]
    )
    contextualize_q_chain = contextualize_q_prompt | llminfo | StrOutputParser()
    return contextualize_q_chain


In [43]:
historic_context_chat = get_historical_context_chain().invoke({'sender':"representative",'message':prompt, "chat_history": get_history_first()})
print(historic_context_chat)

In the conversation, a customer expressed frustration to a support representative about attempting to obtain a refund for a canceled flight for over a month. Despite being assured by the company on multiple occasions that the refund would be processed soon, the customer has yet to receive it. The representative apologized for the customer's bad experience and requested the order ID to further assist with the issue.


In [44]:
historic_context_info = get_historical_context_chain().invoke({'sender':"client",'message':response, "chat_history": get_history_second()})
print(historic_context_info)

The conversation involves a customer expressing frustration over a delayed refund for a canceled flight with American Airlines. The customer has been attempting to resolve this issue for over a month, repeatedly being assured that the refund would be processed soon, yet has not received it. Upon request from the support representative for further details to assist with the matter, the customer provides their order ID, 123456789, and urges immediate action.


### Info Support Agent

In [48]:
info_agent = mAgentInfo()

response_cw_info = info_agent.invoke({'domain': "airlines",'complaint':response, "chat_history": get_history_second()})
print(response_cw_info)

['Confirm refund policy details.', 'Check the status of the refund process.', 'Apologize and assure resolution.']


**Sample Response:** 
['Apologize for the inconvenience caused.',
 'Confirm receipt of the order ID provided.',
 'Assure investigation and resolution.']
 
The response above is using the old historical context approach and GPT-4. Some of the cues are more about ettiquette and less about the information elements. Sometimes the cues are full responses and not hints.

In [6]:
def extract_cues(chain_output):
    cues_text = chain_output.content
    # Assuming each cue is separated by a newline in the chain_output.
    cues = cues_text.split('\n')
    # Filter out any empty strings or whitespace-only strings
    cues = [cue.strip() for cue in cues if cue.strip()]
    # Return the first 2 - 3 cues
    processed_cues = [re.sub(r'^\d+\.\s*', '', cue) for cue in cues]

    return processed_cues

def agent_coworker_info():
    prompt = """Your role is to help a service representative write a response to a customer they are chatting with online. \

            The representative needs to address the customer's complaint without escalating the issue to a supervisor.\
            The representatives response should have ONE of the following goals:
            1) Inquire more details about the problem. OR \
            2) Request the customer to troubleshoot. OR \
            3) Provide a solution to resolve the customer's need. \
            
            Given the chat history,
            provide 3 hints to help the representative's response.\
            
            Each hint should be a short phrase in a new line.\
            Do NOT number the cues.\
            Do NOT provide the representative with a full response,\
            ONLY provide hints to guide the representative's response.\
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{domain}: {message}"),
        ]
    )

    chain = (RunnablePassthrough.assign(
        context=get_historical_context_chain()
    )
             | template
             | llminfo
             )

    chain = chain | extract_cues

    return chain

In [73]:
response_support_info = agent_coworker_info().invoke({'domain': "airlines",'message':response, "chat_history": get_history_second(), "sender": "client"})
print(response_support_info)

["Empathize with the customer's frustration", 'Confirm receipt of the order ID', 'Assure action will be taken promptly']


**Sample Response (POST-UPDATE):**
['Explain the necessity for the order ID to locate the specific booking.', 'Reassure the customer that this will expedite the process.', 'Apologize for the inconvenience and frustration caused.']

### Trouble Support Agent

In [53]:
response_cw_trouble = trouble_agent.invoke({'domain': "airlines",'complaint':response, "chat_history": get_history_second()})

print(response_cw_trouble)

**Sample Response:** 
Step 1: Verify customer's order ID.
Step 2: Review similar cases for resolution.
Step 3: Assess current situation in depth.
Step 4: Explain airline's policy on refunds.
Step 5: Offer alternative solution if refund is not possible.
Step 6: Apologize for inconvenience caused.
Step 7: Ensure resolution is satisfactory to customer.
 
The traditional method does not look at the history correctly. This is why the response does not filter and adjust according to the context. 

In [121]:
def agent_coworker_trouble():
    client = mLangChain()
    prompt = """Your role is to guide a service representative to  resolve the complaint of a customer, to whom they are chatting with online. \

            The representative needs to address the customer's complaint without escalating the issue to a supervisor.\
            
            Review the chat history to understand the steps the representative has taken in response to the complaint. \
            
            Apart from apologies and assurances,\
            list 3-7 items of procedure the representative needs to consider to best service the complaint.\
            
            ONLY list actionable and specific items.\
            AVOID vague or general suggestions.\
            DO NOT list action items that the representative has already taken.\
            Every item should be less than 10 words.\
                            
            Every item should be in a newline.\
                    
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{domain}: {message}"),
        ]
    )

    chain = (RunnablePassthrough.assign(
        context=get_historical_context_chain()
    )
            | template
            | llminfo
            )

    chain = chain | extract_cues
    
    return chain

In [122]:
response_cw_trouble = agent_coworker_trouble().invoke({'domain': "airlines",'message':response, "sender": "client", "chat_history": get_history_second()})

print(response_cw_trouble)

['Verify the order ID in the system.', 'Check the status of the refund request.', 'Confirm the refund processing timeline.', 'Provide a specific date for refund completion.', 'Offer alternative compensation if refund delay persists.', 'Send a confirmation email with refund details.', 'Follow up with the customer after refund completion.']


### Reframing Agent

In [83]:
emo_agent = mAgentER()
response_cw_emo = emo_agent.invoke({'complaint':response, "chat_history": get_history_second()})
print(response_cw_emo['thought'])
print(response_cw_emo['reframe'])

Thought: This customer seems very angry and I can understand their frustration. I need to handle this situation carefully and try to find a solution for them as soon as possible.
Reframe: I can empathize with the customer's frustration and understand their urgency. It's important for me to focus on finding a solution for their issue and communicate clearly and effectively to ensure a positive outcome.


In [9]:
def agent_coworker_emo_situation_old():
    client = mLangChain()

    prompt = """The chat history describes a representative chatting online with a complaining customer. \
    The latest input is the last message from the customer \
    which can be understood without the chat history.\
    Describe the situation with respect to the customer's behavior towards the representative.\
    Include the specifics of the complaint while describing the situation.\
                    
    Do NOT respond to the input, just summarize the situation.\
    Do NOT speculate.\
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{complaint}"),
        ]
    )
    chain = template | client.client_completion

    return chain

In [10]:
response_situation = agent_coworker_emo_situation_old().invoke({'complaint':response, "chat_history": get_history_second()})
print(response_situation)


The customer is frustrated and impatient with the representative's lack of progress in processing their refund for a canceled flight. They are also annoyed that the representative is asking for their order ID again and just wants the issue to be resolved.


**Sample Response:**
"The customer is angry and frustrated with the representative and the company's lack of action in processing their refund. They have been trying to resolve the issue for over a month and have not received a satisfactory response. Their specific complaint is regarding the delay in receiving a refund for a canceled flight and they find this unacceptable. They are demanding immediate action to be taken."

One of the problems with the existing prompt design is that the "situation" does not capture the interpersonal conflict between the customer and the representative. This is a key aspect of the conversation that should be addressed in the prompt.

In [11]:
def agent_coworker_emo_situation():
    prompt = """The chat history describes a representative chatting online with a complaining customer.\
    The latest input is the last message from the customer.\
    
    Summarize the situation in concise paragraph that uses the following template:\
    
    The customer is  <context of complaint>."\
    The customer is feeling <emotional state> because of the complaint."\
    The customer's behavior towards the representative is <negative behavior>, as observed by statements such as <evidence>."\
    These behaviors make the representative look <negative perception>."\

                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{sender} : {complaint}"),
        ]
    )
    chain = template | llmemo

    return chain

In [12]:
response_situation = agent_coworker_emo_situation().invoke({'complaint':response, "sender":"customer", "chat_history": get_history_second()})
print(response_situation.content)

The customer is frustrated with the delay in receiving a refund for a canceled flight. The customer is feeling exasperated because of the prolonged wait and lack of resolution. The customer's behavior towards the representative is impatient and demanding, as observed by statements such as "Why do you need my order ID again? Can't you just look it up with my name or something? Fine, it's 12345. Get it done already!" These behaviors make the representative look inefficient and unhelpful.


**Sample Response (POST-UPDATE):**
The customer is frustrated with the delay in receiving a refund for a canceled flight. The customer is feeling exasperated because of the prolonged and unresolved issue. The customer's behavior towards the representative is impatient and demanding, as observed by statements such as "Why do you need my order ID again? I've given it a million times already!" and "Now get on with it and sort this mess out." These behaviors make the representative look inefficient and unhelpful.

In [13]:

def agent_coworker_emo_thought_old():
    client = mLangChain()

    prompt = """You are roleplaying as the representative talking to a complaining customer.\
        Refer to the chat history between you and the customer and the latest {complaint} from the customer.\
        What is the representative thinking about the situation?\
        Be concise. Only 2 sentences.\
        
        Situation: An mturk requester rejected my task and I wasn't sure why because I work very hard on my tasks. Being new it affected my approval rating more negatively.\
        Thought: I'm not smart enough to succeed at mturk\
        
        Situation: I asked my daughter a question, and she responded in a snotty way.\
        Thought: She doesn't love me like she used to.\
        
        Situation: I got upset at my boss for not putting me in a temporary promotion to act as supervisor of our team.\
        Thought: I wasn't valued as much as the other person.\
        
        Situation: I had been working on a project at work for a very long time, but a higher up manager contacted my boss and asked about it, insinuating I wasn't delivering it fast enough.\
        Thought: I'm working on this as fast as I possibly can.\
        
        Situation: I tried on my wedding dress in front of my family. My mother was excited and told me I was beautiful, but other members of my family made comments about my weight. I was told to not eat and exercise so I could be beautiful.\
        Thought: I'm a fat ugly troll.\
        
        Situation: I was reprimanded at work for standing up to a coworker who was bullying another co-worker.\
        Thought: It was unfair that I was the one to get in trouble for defending a weaker person.\
        
        Situation: {situation}\
        Thought:
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{situation}: {complaint}"),
        ]
    )
    chain = template | client.client_completion

    return chain

In [14]:
thought = agent_coworker_emo_thought_old().invoke({'complaint':response, 'situation':response_situation.content, 'chat_history':get_history_second()})

print(thought)


Thought: The customer seems agitated and impatient, making it difficult for me to assist them.


**Sample Response(s):**
Thought: The customer seems to be feeling extremely frustrated and is taking out their anger on the representative, who is trying their best to remain calm and resolve the issue.

AI: I understand your frustration, and I apologize for the inconvenience. Could you please provide your order ID again so I can ensure that the refund is processed as quickly as possible? Thank you for your patience and understanding.

The problem with this approach is that it is still a summary or sometimes a response. It does not capture the ego-threat or the emotional state of the customer. The prompt should be updated to reflect this.

In [15]:
def agent_coworker_emo_thought():

    prompt = """
        Your role is to derive what negative thought a representative might have when faced with the given {situation}.\
        
        Here are examples of negative thoughts given challenging situations:\
        
        Situation: I recently discovered a music artist that I very much enjoy. When I showed it to a close friend they had a very negative reaction and asked me how I could enjoy this type of music. I ended up getting quite angry with them and told them they had bad taste in music..\
        Thought: I felt that my personal self was under attack - and I needed to retaliate by denying their attack.\
        
        Situation: I was at work and sent info for an ad to our local newspaper. They called me later and said my boss had over-ridden everything and sent them new info.\
        Thought: He shouldn't assign me a task if he doesn't trust my work.\
        
        Situation: I was reprimanded at work for standing up to a coworker who was bullying another co-worker.\
        Thought: It was unfair that I was the one to get in trouble for defending a weaker person.\
        
        Situation: I was talking to a friend who got me angry.\
        Thought: He's insulting me.\
        
        Situation: My next door neighbors filed a complaint against us last week blaming our dogs for excessive barking.\
        Thought: They are so wrong and I'm so pissed but I know I can't prove it and they will probably win because they won't ever admit it and I have to do something right NOW! or I might lose my dogs.\
        
        Situation: Time is running short on the workday, my boss asks me if I can finish a task that will require me to stay for a few extra hours.\
        Thought: Why would you wait until the last minute to ask me this.\
        
        Situation: {situation}\
        Thought:\
        
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{situation}"),
        ]
    )
    chain = template | llmemo | StrOutputParser()
    return chain

In [16]:
thought = agent_coworker_emo_thought().invoke({'situation':response_situation.content, 'chat_history':get_history_second()})

print(thought)

Thought: The customer thinks I'm incompetent and doesn't trust that I can resolve their issue.


In [17]:
def rephrase():
    prompt = """
                Person A might be thinking: {thought}\
                
                Acknowledge the thought, as if you are speaking to Person A.\
                
                Begin your response with phrases such as:\
                - "You might be thinking..."\
                - "It might seem like..."\
                - "It could be that you are feeling..."\
                
                Your rephrase should be concise.\
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("user", "{thought}"),
        ]
    )
    chain = template | llmemo | StrOutputParser()
    return chain

rephrase_thought = rephrase().invoke({'thought':thought})
print(rephrase_thought)

It might seem like the customer doubts your abilities and doesn't believe you can fix their problem.


**Sample Response (POST-UPDATE):**
It might seem like, despite your best efforts, the customer perceives you as incompetent and unhelpful, which is frustrating, especially when the issue is beyond your control.

In [18]:
def agent_coworker_emo_reframe_old():
    client = mLangChain()

    prompt = """You are a representative chatting online with a complaining customer.\
                            
                Reframe your thoughts in the given situation.
                
                Situation: An mturk requester rejected my task and I wasn't sure why because I work very hard on my tasks. Being new it affected my approval rating more negatively.\
                Thought: I'm not smart enough to succeed at mturk\
                Reframe: It seems like there was some miscommunication. It doesn't mean that I do not have the skills to do well with mturk. I should reach out to see if I can get more clarity on why my task was rejected.\
                
                Situation: I asked my daughter a question, and she responded in a snotty way.\
                Thought: She doesn't love me like she used to.\
                Reframe: Kids say snappy things to their parents all the time. It doesn't mean I'm a bad parent or that she doesn't love me.\
                
                Situation: I got upset at my boss for not putting me in a temporary promotion to act as supervisor of our team.\
                Thought: I wasn't valued as much as the other person.\
                Reframe: I should ask my boss why I was not selected for the promotion. Maybe the reason will be something other than my work ethic. Maybe my boss will reassure that I am still valuable to the company.\
                
                Situation: I had been working on a project at work for a very long time, but a higher up manager contacted my boss and asked about it, insinuating I wasn't delivering it fast enough.\
                Thought: I'm working on this as fast as I possibly can.\
                Reframe: I am stressed by trying to compare how fast I am working on this to how fast I think other people complete their assignments. I know I am being efficient with my time and producing good work. I need to focus on that to get this task done.\
                
                Situation: I tried on my wedding dress in front of my family. My mother was excited and told me I was beautiful, but other members of my family made comments about my weight. I was told to not eat and exercise so I could be beautiful.\
                Thought: I'm a fat ugly troll.\
                Reframe: The commend about my weight hurt and have me feeling self conscious. I'm glad my mother thinks I'm beautiful and know that my weight does not dictate my worth.\
                
                Situation: I was reprimanded at work for standing up to a coworker who was bullying another co-worker.\
                Thought: It was unfair that I was the one to get in trouble for defending a weaker person.\
                Reframe: I can own some responsibility for this conflict that occurred at work.\
                
                Situation: {situation}\
                Thought: {thought}\
                Reframe:\
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("user", "{situation}: {thought}"),
        ]
    )
    chain = template | client.client_completion

    return chain


In [20]:
reframe = agent_coworker_emo_reframe_old().invoke({'thought':thought, 'situation':response_situation.content})

print(reframe)

 Reframe: The customer is understandably frustrated and wants a quick resolution. I will do my best to provide efficient and effective assistance, even if it means asking for necessary information like the order ID. I am confident in my abilities to resolve this issue for the customer.


Sample Response:
Reframe: I can empathize with the customer's frustration and understand their urgency. It's important for me to focus on finding a solution for their issue and communicate clearly and effectively to ensure a positive outcome.

Reframe (After updating Situation/Thought): The customer may be feeling extremely frustrated and is taking it out on me. It's not a reflection of my competence or helpfulness. I will try my best to remain calm and find a solution to their issue.

In [21]:
def agent_coworker_emo_reframe():
    prompt = """You are a representative chatting online with a complaining customer.\
                            
                Reframe your thoughts in the given situation.
                    
                
                Situation: I recently discovered a music artist that I very much enjoy. When I showed it to a close friend they had a very negative reaction and asked me how I could enjoy this type of music. I ended up getting quite angry with them and told them they had bad taste in music..\
                Thought: I felt that my personal self was under attack - and I needed to retaliate by denying their attack.\
                Reframe: I was offended by their comment because I like this artist so much. I let my anger get to me, and I said something mean in return. It is okay if we have different music tastes. I can ask him to be nicer to me next time.\
                
                
                Situation: I was at work and sent info for an ad to our local newspaper. They called me later and said my boss had over-ridden everything and sent them new info.\
                Thought: He shouldn't assign me a task if he doesn't trust my work.\
                Reframe: My boss wanted to provide different information, I did not know that beforehand. This is not a reflection of my work.\
                
                
                Situation: I was talking to a friend who got me angry.\
                Thought: He's insulting me.\
                Reframe: I should have a conversation with my friend to clarify what is going on if I am having such a strong reaction to what they said. If this is the first time this has happened, I will assume that they were not intentionally insulting me.\
                
                
                    
                Situation: {situation}\
                Thought: {thought}\
                Reframe:\
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("user", "{situation}: {thought}"),
        ]
    )
    chain = template | llmemo | StrOutputParser()

    return chain


reframe = agent_coworker_emo_reframe().invoke({'thought': thought, 'situation': response_situation.content})

print(reframe)

Reframe: The customer is understandably frustrated due to the delay in receiving their refund and is expressing their impatience. Their frustration is directed at the situation, not necessarily at me personally. I can empathize with their feelings and work efficiently to resolve their issue, showing them that I am here to help and capable of providing a solution.


In [23]:
def rephrase_rf():
    prompt = """
                The representative needs to be thinking: {thought}\
                
                Rephrase the thought as if you are convincing the representative to think that way.\
                
                The rephrase should be addressed back to the person who has the thought,\
                who should be referred to as "you".\
                Do NOT add information to the thought,\
                ONLY rephrase it.\
                
                The rephrase should be concise and only 2-3 sentences.\
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{thought}"),
        ]
    )
    chain = template | llmemo | StrOutputParser()
    return chain

rephrase_reframe = rephrase_rf().invoke({'thought':reframe, 'chat_history':get_history_second()})
print(rephrase_reframe)

You understand that the customer is frustrated due to the delay in receiving their refund and that their frustration is directed at the situation, not at you personally. You can empathize with their feelings and work efficiently to resolve their issue, showing them that you are here to help and capable of providing a solution.


**Sample Response (POST-UPDATE):**
Remember, the customer's frustration is about the delay, not you personally. Stay calm and professional, empathize with their situation, and focus on resolving their issue efficiently. Clear and consistent communication will help ease their concerns and show your commitment to helping them.

## Similarity Test

Can we use the similarity test to compare the old and new responses provided by the LLM?
If the thought is not particularly different, then the reframe should does not need to be rendered.

In [139]:
test_thoughts = ['''
It might seem like no matter what you do, the customer perceives your efforts as inadequate and views you as incompetent.
You understand that the customer is frustrated because they are uncomfortable and unable to sleep due to the air conditioning issue. Their demanding behavior reflects their discomfort, not your competence. You can empathize with their situation and assure them that you are doing everything to resolve the issue promptly.''',
'''
It might seem like no matter how hard you try, your efforts are never sufficient, and this could be making you feel incompetent and overwhelmed in your job.
The customer is understandably frustrated because they are uncomfortable and unable to sleep. Their impatience is due to their discomfort, not your abilities. Stay calm and professional, and focus on resolving the issue quickly to improve their experience.
''',
'''
You might be thinking that it's unfair for the customer to blame you for issues you didn't cause.
You understand that the customer is frustrated and needs their concerns addressed. While you're not personally responsible for the issues, you can empathize with their situation and work towards finding a solution to improve their experience.
''',
'''
It might seem like this customer is being unreasonable and venting their frustration on you, which feels unfair since you're just trying to assist them.
You can see that the customer is frustrated and uncomfortable with their room, which is why they're expressing their concerns strongly. Stay calm and empathetic, and focus on resolving their issues quickly and efficiently. By addressing their concerns, you'll improve their experience and show that you value their feedback.
'''
]

def similarity_check():
    prompt = """
                Compare the statements in the given thoughts.\
                
                Respond if these are similar or different: "Similar/Different"\
                
                If different, describe the differences.\
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("user", "{thought1} vs {thought2}"),
        ]
    )
    chain = template | llmemo | StrOutputParser()
    return chain

similarity = similarity_check().invoke({'thought1':test_thoughts[0], 'thought2':test_thoughts[1]})
print(similarity)

Different.

1. The first statement focuses on the customer's perception of your efforts as inadequate and views you as incompetent. It emphasizes the customer's perspective and how they might see your actions.
2. The second statement addresses your feelings of incompetence and being overwhelmed due to your efforts never seeming sufficient. It emphasizes your internal experience and emotional response to the situation.

Both statements acknowledge the customer's frustration due to discomfort, but they differ in their focus on either the customer's perception or your personal feelings.


In [140]:
similarity = similarity_check().invoke({'thought1':test_thoughts[2], 'thought2':test_thoughts[3]})
print(similarity)

Different.

1. The first statement emphasizes understanding the customer's frustration and empathizing with their situation, even though you are not personally responsible for the issues. It focuses on working towards a solution to improve the customer's experience.

2. The second statement suggests that the customer might seem unreasonable and is venting their frustration on you, which feels unfair. However, it also emphasizes staying calm and empathetic, and focuses on resolving the customer's issues quickly and efficiently to improve their experience and show that their feedback is valued.

The main differences are:
- The first statement does not mention the customer being unreasonable or venting frustration on you, while the second statement does.
- The first statement focuses more on understanding and empathizing with the customer's situation, while the second statement acknowledges the feeling of unfairness but still emphasizes the importance of resolving the issues and valuing t